In [57]:
from stock_data import get_hs300_sample, get_stock_data, get_stock_name
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import backtrader as bt
plt.rcParams['font.sans-serif'] = ['SimHei']
plt.rcParams['axes.unicode_minus'] = False
import yaml
# 读取配置文件
config = yaml.load(open('config.yaml', 'r'), Loader=yaml.FullLoader)
# 获取数据
amount = 1
start_date = '20211115'
end_date = '20241115'
# data = get_hs300_sample(stock_count=1, start_date=start_date, end_date=end_date)
# test_data = data[list(data.keys())[0]] # 取出第一只股票的数据
id = config['stock_list'][0]['id']
name = get_stock_name(id)
print(f'{id} {name}')
test_data = get_stock_data(id, start_date=start_date, end_date=end_date)
# 计算出区间涨幅作为起始的收益率基准
start_price = test_data['close'].iloc[0]
end_price = test_data['close'].iloc[-1]
base_return = (end_price - start_price) / start_price
print(f'区间涨幅: {base_return:.2%}')
print(test_data.head())

601857 中国石油
区间涨幅: 66.94%
            open  close  high   low   volume  turnover
datetime                                              
2021-11-15  4.80   4.81  4.83  4.76  1231063      0.08
2021-11-16  4.81   4.77  4.85  4.77  1725119      0.11
2021-11-17  4.77   4.83  4.83  4.74  1275661      0.08
2021-11-18  4.78   4.77  4.81  4.76  1368661      0.08
2021-11-19  4.77   4.83  4.85  4.74  1502051      0.09


In [58]:
import backtrader as bt
import backtrader.feeds as btfeeds
import backtrader.indicators as btind
import backtrader.analyzers as btanalyzers
import backtrader.strategies as btstrats
import yaml

# 读取初始资金配置
config = yaml.load(open('config.yaml', 'r'), Loader=yaml.FullLoader)
initial_cash = config['user']['initial_cash']

# 读取仓位比例配置
max_position_aggressive = config['position_ratio']['max_aggressive']
max_position_conservative = config['position_ratio']['max_conservative']
min_position_aggressive = config['position_ratio']['min_aggressive']
min_position_conservative = config['position_ratio']['min_conservative']

# 双均线策略
# 基准策略A1 - 保守的双均线策略 MA20 MA50
# 基准策略A2 - 激进的双均线策略 MA5 MA10

# RSI策略
# 基准策略B1 - 保守的RSI策略 RSI<30买入，RSI>70卖出
# 基准策略B2 - 激进的RSI策略 RSI<40买入，RSI>60卖出

# 波动率策略 使用10天ATR作为波动率指标
# 基准策略C1 - 保守的波动率策略 收盘价高于前一天高点 + 1.0×ATR 买入，收盘价低于前一天低点 - 1.0×ATR 卖出
# 基准策略C2 - 激进的波动率策略 收盘价高于前一天高点 + 0.5×ATR 买入，收盘价低于前一天低点 - 0.5×ATR 卖出

class BaseConservativeStrategy(bt.Strategy):
    params = (
        ('max_position_ratio', 0.7),  # 默认值，可在子类中覆盖
        ('min_position_ratio', 0.1),  # 默认值，可在子类中覆盖
        ('adjust_ratio', 0.1),        # 每次调整仓位的比例
        ('min_trade_size', 1),        # 最小交易单位
    )

    def __init__(self):
        self.max_position_ratio = self.params.max_position_ratio
        self.min_position_ratio = self.params.min_position_ratio
        self.adjust_ratio = self.params.adjust_ratio
        self.min_trade_size = self.params.min_trade_size

    def get_position_ratio(self):
        """计算当前仓位比例"""
        total_value = self.broker.getvalue()  # 当前账户总资产
        position_value = self.broker.getposition(self.data).size * self.data.close[0]
        return position_value / total_value

    def dynamic_buy(self):
        """动态加仓逻辑"""
        position_ratio = self.get_position_ratio()
        if position_ratio < self.max_position_ratio:
            available_cash = self.broker.getcash()
            max_investment = self.broker.getvalue() * self.max_position_ratio
            target_investment = position_ratio * self.broker.getvalue() + (max_investment - position_ratio * self.broker.getvalue()) * self.adjust_ratio
            investment = min(target_investment, available_cash)
            size = int(investment // self.data.close[0])
            if size >= self.min_trade_size:
                self.buy(size=size)

    def dynamic_sell(self):
        """动态减仓逻辑"""
        position_ratio = self.get_position_ratio()
        if position_ratio > self.min_position_ratio:
            position_value = self.broker.getposition(self.data).size * self.data.close[0]
            min_investment = self.broker.getvalue() * self.min_position_ratio
            target_investment = position_value - (position_value - min_investment) * self.adjust_ratio
            size = int((position_value - target_investment) // self.data.close[0])
            if size >= self.min_trade_size:
                self.sell(size=size)


class BaseAggressiveStrategy(bt.Strategy):
    params = (
        ('max_position_ratio', 1.0),  # 默认值，可在子类中覆盖
        ('min_position_ratio', 0.3),  # 默认值，可在子类中覆盖
        ('adjust_ratio', 0.2),        # 每次调整仓位的比例
        ('min_trade_size', 1),        # 最小交易单位
    )

    def __init__(self):
        self.max_position_ratio = self.params.max_position_ratio
        self.min_position_ratio = self.params.min_position_ratio
        self.adjust_ratio = self.params.adjust_ratio
        self.min_trade_size = self.params.min_trade_size

    def get_position_ratio(self):
        """计算当前仓位比例"""
        total_value = self.broker.getvalue()  # 当前账户总资产
        position_value = self.broker.getposition(self.data).size * self.data.close[0]
        return position_value / total_value

    def dynamic_buy(self):
        """动态加仓逻辑"""
        position_ratio = self.get_position_ratio()
        if position_ratio < self.max_position_ratio:
            available_cash = self.broker.getcash()
            max_investment = self.broker.getvalue() * self.max_position_ratio
            target_investment = position_ratio * self.broker.getvalue() + (max_investment - position_ratio * self.broker.getvalue()) * self.adjust_ratio
            investment = min(target_investment, available_cash)
            size = int(investment // self.data.close[0])
            if size >= self.min_trade_size:
                self.buy(size=size)

    def dynamic_sell(self):
        """动态减仓逻辑"""
        position_ratio = self.get_position_ratio()
        if position_ratio > self.min_position_ratio:
            position_value = self.broker.getposition(self.data).size * self.data.close[0]
            min_investment = self.broker.getvalue() * self.min_position_ratio
            target_investment = position_value - (position_value - min_investment) * self.adjust_ratio
            size = int((position_value - target_investment) // self.data.close[0])
            if size >= self.min_trade_size:
                self.sell(size=size)




class DualMovingAverageConservative(BaseConservativeStrategy):
    params = (
        ('sma1', 20),
        ('sma2', 50),
        ('max_position_ratio', max_position_conservative),
        ('min_position_ratio', min_position_conservative),
    )

    def __init__(self):
        super().__init__()
        self.sma1 = btind.SimpleMovingAverage(self.data, period=self.params.sma1)
        self.sma2 = btind.SimpleMovingAverage(self.data, period=self.params.sma2)
        self.crossover = btind.CrossOver(self.sma1, self.sma2)

    def next(self):
        if self.crossover > 0:
            self.dynamic_buy()
        elif self.crossover < 0:
            self.dynamic_sell()
            
class DualMovingAverageAggressive(BaseAggressiveStrategy):
    params = (
        ('sma1', 5),
        ('sma2', 10),
        ('max_position_ratio', max_position_aggressive),
        ('min_position_ratio', min_position_aggressive),
    )

    def __init__(self):
        super().__init__()
        self.sma1 = btind.SimpleMovingAverage(self.data, period=self.params.sma1)
        self.sma2 = btind.SimpleMovingAverage(self.data, period=self.params.sma2)
        self.crossover = btind.CrossOver(self.sma1, self.sma2)

    def next(self):
        if self.crossover > 0:
            self.dynamic_buy()
        elif self.crossover < 0:
            self.dynamic_sell()
            
class RSIConservative(BaseConservativeStrategy):
    params = (
        ('rsi_low', 30),
        ('rsi_high', 70),
        ('max_position_ratio', max_position_conservative),
        ('min_position_ratio', min_position_conservative),
    )

    def __init__(self):
        super().__init__()
        self.rsi = btind.RSI(self.data)
        
    def next(self):
        if self.rsi < self.params.rsi_low:
            self.dynamic_buy()
        elif self.rsi > self.params.rsi_high:
            self.dynamic_sell()
            
class RSIAggressive(BaseAggressiveStrategy):
    params = (
        ('rsi_low', 40),
        ('rsi_high', 60),
        ('max_position_ratio', max_position_aggressive),
        ('min_position_ratio', min_position_aggressive),
    )

    def __init__(self):
        super().__init__()
        self.rsi = btind.RSI(self.data)
        
    def next(self):
        if self.rsi < self.params.rsi_low:
            self.dynamic_buy()
        elif self.rsi > self.params.rsi_high:
            self.dynamic_sell()
            
class VolatilityConservative(BaseConservativeStrategy):
    params = (
        ('atr_period', 10),
        ('atr_multiplier', 1.0),
        ('max_position_ratio', max_position_conservative),
        ('min_position_ratio', min_position_conservative),
    )

    def __init__(self):
        super().__init__()
        self.atr = btind.ATR(self.data, period=self.params.atr_period)
        
    def next(self):
        if self.data.close > self.data.high[-1] + self.atr * self.params.atr_multiplier:
            self.dynamic_buy()
        elif self.data.close < self.data.low[-1] - self.atr * self.params.atr_multiplier:
            self.dynamic_sell()
            
class VolatilityAggressive(BaseAggressiveStrategy):
    params = (
        ('atr_period', 10),
        ('atr_multiplier', 0.5),
        ('max_position_ratio', max_position_aggressive),
        ('min_position_ratio', min_position_aggressive),
    )

    def __init__(self):
        super().__init__()
        self.atr = btind.ATR(self.data, period=self.params.atr_period)
        
    def next(self):
        if self.data.close > self.data.high[-1] + self.atr * self.params.atr_multiplier:
            self.dynamic_buy()
        elif self.data.close < self.data.low[-1] - self.atr * self.params.atr_multiplier:
            self.dynamic_sell()

In [59]:
# 策略列表
strategies = [
    ("DualMovingAverageConserative", DualMovingAverageConservative),
    ("DualMovingAverageAggressive", DualMovingAverageAggressive),
    ("RSIConserative", RSIConservative),
    ("RSIAggressive", RSIAggressive),
    ("VolatilityConserative", VolatilityConservative),
    ("VolatilityAggressive", VolatilityAggressive),
]

# 存储结果
results = []

# 单独运行每个策略
for strategy_name, strategy_class in strategies:
    # 创建 Cerebro 实例
    cerebro = bt.Cerebro()
    
    # 加载数据
    data = btfeeds.PandasData(dataname=test_data)
    cerebro.adddata(data)
    
    # 添加策略
    cerebro.addstrategy(strategy_class)
    
    # 设置初始资金
    cerebro.broker.set_cash(initial_cash)
    
    # 设置手续费
    cerebro.broker.setcommission(commission=0.0005)
    
    # 添加分析器
    cerebro.addanalyzer(bt.analyzers.SharpeRatio, _name='sharpe')
    cerebro.addanalyzer(bt.analyzers.DrawDown, _name='drawdown')
    cerebro.addanalyzer(bt.analyzers.Returns, _name='returns')
    
    # 运行回测
    result = cerebro.run()
    strat = result[0]
    
    # 获取结果
    final_value = cerebro.broker.getvalue()
    sharpe = strat.analyzers.sharpe.get_analysis()
    drawdown = strat.analyzers.drawdown.get_analysis()
    returns = strat.analyzers.returns.get_analysis()
    
    # 保存结果
    results.append({
        'strategy': strategy_name,
        'final_value': final_value,
        'sharpe': sharpe,
        'drawdown': drawdown,
        'returns': returns,
        'cerero_instance': cerebro,
    })

# 输出结果
for res in results:
    print(f"策略: {res['strategy']}")
    print(f"初始资金: {initial_cash:.2f}")
    print(f"结束资金: {res['final_value']:.2f}")
    print(f"夏普比率: {res['sharpe']}")
    print(f"最大回撤: {res['drawdown']}")
    print(f"收益率: {res['returns']}")
    print("-" * 50)
    cerebro_res = res['cerero_instance']
    fig = cerebro_res.plot(style='candle')
    show = fig[0][0]
    show.set_size_inches(30,15)
    show.savefig(f'Result_{res["strategy"]}.png')
    
    

策略: DualMovingAverageConserative
初始资金: 1000000.00
结束资金: 1429619.10
夏普比率: OrderedDict({'sharperatio': 0.6524369313080108})
最大回撤: AutoOrderedDict({'len': 141, 'drawdown': 17.66627331682318, 'moneydown': 306752.0783500001, 'max': AutoOrderedDict({'len': 192, 'drawdown': 19.28586517214726, 'moneydown': 334874.20455000014})})
收益率: OrderedDict({'rtot': 0.3574080458993119, 'ravg': 0.0004902716679002907, 'rnorm': 0.1315048362896074, 'rnorm100': 13.15048362896074})
--------------------------------------------------


<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

策略: DualMovingAverageAggressive
初始资金: 1000000.00
结束资金: 1253609.63
夏普比率: OrderedDict({'sharperatio': 0.5604149412923976})
最大回撤: AutoOrderedDict({'len': 141, 'drawdown': 16.008272670698535, 'moneydown': 238929.7780149998, 'max': AutoOrderedDict({'len': 201, 'drawdown': 17.297247837972094, 'moneydown': 258168.24033499975})})
收益率: OrderedDict({'rtot': 0.22602709322034636, 'ravg': 0.0003100508823324367, 'rnorm': 0.08126626545672358, 'rnorm100': 8.126626545672359})
--------------------------------------------------


<IPython.core.display.Javascript object>

...

<IPython.core.display.Javascript object>

策略: VolatilityConserative
初始资金: 1000000.00
结束资金: 1247750.06
夏普比率: OrderedDict({'sharperatio': 0.43485203051348725})
最大回撤: AutoOrderedDict({'len': 141, 'drawdown': 20.344405276391072, 'moneydown': 318681.10462500015, 'max': AutoOrderedDict({'len': 194, 'drawdown': 23.030651665295952, 'moneydown': 360759.3052350001})})
收益率: OrderedDict({'rtot': 0.22134197791897275, 'ravg': 0.0003036241123717047, 'rnorm': 0.0795165222463067, 'rnorm100': 7.951652224630671})
--------------------------------------------------


<IPython.core.display.Javascript object>

策略: VolatilityAggressive
初始资金: 1000000.00
结束资金: 1382386.26
夏普比率: OrderedDict({'sharperatio': 0.5715763076406506})
最大回撤: AutoOrderedDict({'len': 87, 'drawdown': 19.868185993662852, 'moneydown': 342754.0947050003, 'max': AutoOrderedDict({'len': 259, 'drawdown': 21.921518702162142, 'moneydown': 378176.96591500007})})
收益率: OrderedDict({'rtot': 0.3238111811674344, 'ravg': 0.00044418543370018437, 'rnorm': 0.11843985690175209, 'rnorm100': 11.84398569017521})
--------------------------------------------------


<IPython.core.display.Javascript object>

In [60]:
import matplotlib.pyplot as plt
import numpy as np

# 策略数据
strategy_names = [res['strategy'] for res in results]
returns = [res['returns']['rtot'] * 100 for res in results]  # 转换为百分比
max_drawdowns = [res['drawdown']['max']['drawdown'] for res in results]
sharpe_ratios = [res['sharpe']['sharperatio'] for res in results]

# 保存数值到csv
df = pd.DataFrame({
    '策略': strategy_names,
    '收益率 (%)': returns,
    '最大回撤 (%)': max_drawdowns,
    '夏普比率': sharpe_ratios,
})
df.to_csv(f'{id}.csv', index=False)

In [66]:
def plot_results(data, id, name):
    # 提取数据
    strategy_names = data['策略']
    returns = data['收益率 (%)']
    max_drawdowns = data['最大回撤 (%)']
    sharpe_ratios = data['夏普比率']

    # 设置图表布局
    x = np.arange(len(strategy_names))  # 策略数量
    width = 0.25  # 柱状图宽度

    fig, ax = plt.subplots(figsize=(12,8))

    # 绘制柱状图
    bar1 = ax.bar(x - width, returns, width, label='收益率 (%)', color='tab:blue')
    bar2 = ax.bar(x, max_drawdowns, width, label='最大回撤 (%)', color='tab:orange')
    bar3 = ax.bar(x + width, sharpe_ratios, width, label='夏普比率', color='tab:green')

    # 添加数值标签
    def add_labels(bars):
        for bar in bars:
            height = bar.get_height()
            ax.text(bar.get_x() + bar.get_width() / 2, height,
                    f'{height:.2f}', ha='center', va='bottom')

    add_labels(bar1)
    add_labels(bar2)
    add_labels(bar3)
    
    # 添加一条基准线 表示区间涨幅
    ax.axhline(y=base_return * 100, color='r', linestyle='--', label='区间涨幅')    

    # 添加标题和标签
    ax.set_title(f'{id} {name}', fontsize=16)
    ax.set_xlabel('策略', fontsize=12)
    ax.set_ylabel('指标值', fontsize=12)
    ax.set_xticks(x)
    ax.set_xticklabels(strategy_names, rotation=45, ha='right')
    ax.legend()

    # 调整布局并显示图表
    plt.tight_layout()
    plt.savefig(f'{id}.png')
    plt.show()

In [68]:
results_df = pd.read_csv(f'{id}.csv')
plot_results(results_df, id, name)

<IPython.core.display.Javascript object>